# Convert MLD output to MDM output and visualize it

## MDM output format

In [3]:
import numpy as np

mdm_data = np.load('/home/ctq566/motion-diffusion-model/save/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10_The_man_spun_in_place_three_times/results.npy', allow_pickle=True).item()
# print all keys and value type of mdm_data, if it has shape attribute, print its shape in the same line, otherwise print its value
for key, value in mdm_data.items():
    if hasattr(value, 'shape'):
        print(key, type(value), value.shape)
    else:
        print(key, type(value), value)
        

motion <class 'numpy.ndarray'> (3, 22, 3, 120)
text <class 'list'> ['The man spun in place three times.', 'The man spun in place three times.', 'The man spun in place three times.']
lengths <class 'numpy.ndarray'> (3,)
num_samples <class 'int'> 1
num_repetitions <class 'int'> 3


## MLD output format

In [4]:
mld_data = np.load('/home/ctq566/motion-diffusion-model/playground/Example_50_batch0_0.npy', allow_pickle=True)
mld_data.shape


(50, 22, 3)

In [7]:
mld_data_dict = {'motion': mld_data.transpose((1, 2, 0)).reshape((1, 22, 3, 50)), 'text': 'No text', 'lengths': np.asarray([50]), 'num_samples': 1, 'num_repetitions': 1}
np.save('results.npy', mld_data_dict)


## The conversion and the stick animation generation

In [2]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 66.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00


In [1]:
import os
import sys
sys.path.append('/home/pjr726/motion-diffusion-model/')
import numpy as np
import data_loaders.humanml.utils.paramUtil as paramUtil
from data_loaders.humanml.utils.plot_script import plot_3d_motion

def mld_to_mdm(mld_data_path):
    assert isinstance(mld_data_path, str)
    mld_data = np.load(mld_data_path, allow_pickle=True)
    lengths = mld_data.shape[0]
    mld_data_dict = {'motion': mld_data.transpose((1, 2, 0)).reshape((1, 22, 3, lengths)), 'text': 'No text', 'lengths': np.asarray([lengths]), 'num_samples': 1, 'num_repetitions': 1}
    # save the result to the same directory as results.npy
    np.save(f'{os.path.dirname(mld_data_path)}/results.npy', mld_data_dict)

    animation_save_path = f'{os.path.dirname(mld_data_path)}/sample00_rep00.mp4'
    skeleton = paramUtil.t2m_kinematic_chain
    motion = mld_data
    dataset = 'humanml'
    title = ''
    fps = 20
    plot_3d_motion(animation_save_path, skeleton, motion, title, dataset, fps=fps)

    
# mld_to_mdm('/home/pjr726/motion-latent-diffusion/results/mld/1222_PELearn_Diff_Latent1_MEncDec49_MdiffEnc49_bs64_clip_uncond75_01/random_test/random_196_5.npy')


## Batch conversion

In [2]:
import glob
import tqdm


def batch_convert_mld_to_mdm(mld_data_dir):
    assert isinstance(mld_data_dir, str)
    assert os.path.isdir(mld_data_dir)
    print(f'Converting mld data in {mld_data_dir} to mdm data')
    for mld_data_path in tqdm.tqdm(glob.glob(f'{mld_data_dir}/*.npy')):
        # create a new directory for each mld data
        new_dir = f'{mld_data_dir}/{os.path.basename(mld_data_path).split(".")[0]}'
        os.mkdir(new_dir)
        # move the mld data to the new directory
        os.rename(mld_data_path,
                  f'{new_dir}/{os.path.basename(mld_data_path)}')
        # change the mld_data_path to the new directory
        mld_data_path = f'{new_dir}/{os.path.basename(mld_data_path)}'
        # convert the mld data to mdm data
        mld_to_mdm(mld_data_path)
    print('Done')


batch_convert_mld_to_mdm(
    '/home/pjr726/motion-latent-diffusion/results/mld/now_PELearn_Diff_Latent1_MEncDec49_MdiffEnc49_bs64_clip_uncond75_01/samples_2023-05-11-21-50-11')


Converting mld data in /home/pjr726/motion-latent-diffusion/results/mld/now_PELearn_Diff_Latent1_MEncDec49_MdiffEnc49_bs64_clip_uncond75_01/samples_2023-05-11-21-50-11 to mdm data


100%|██████████| 30/30 [00:30<00:00,  1.00s/it]

Done


In [4]:
!pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.8 MB)


In [8]:
import cv2
import math
# Open the video file
cap = cv2.VideoCapture('/home/pjr726/motion-latent-diffusion/results/mld/now_PELearn_Diff_Latent1_MEncDec49_MdiffEnc49_bs64_clip_uncond75_01/samples_2023-05-11-21-50-11/Example_120_batch0_2_5/sample00_rep00.mp4')
# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Set the number of frames you want to extract
num_frames = 12
# Calculate the frame interval
frame_interval = math.floor(total_frames / num_frames)
# Initialize an empty list to store the frames
frames = []
# Loop through the video frames and extract the frames at the specified interval
for i in range(0, total_frames, frame_interval):
    # Set the frame index
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    # Read the frame
    ret, frame = cap.read()
    # Remove the white border (assuming a 5-pixel border)
    # frame = cv2.copyMakeBorder(frame, 30, 20, 20, 30, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    h, w, _ = frame.shape
    # The crop size is 200x100 for MDM single output only
    cropped = frame[(h-150)//2:(h-200)//2+200, (w-100)//2:(w-100)//2+100, :]
    # Add the frame to the list
    frames.append(cropped)
# Concatenate the frames horizontally
frame_array = cv2.hconcat(frames)
# Save the concatenated frames as a single image with higher resolution
cv2.imwrite('motion_boxing.png', frame_array)

True